# 🚀 EnfoadsIA Backend - Google Colab (GPU T4)

Este notebook prepara y ejecuta el servidor backend de EnfoadsIA en una instancia de Google Colab, utilizando una GPU T4 y Ngrok para el túnel público.

### Pasos:
1. **Configuración de Entorno**: Instala dependencias y clona el repositorio.
2. **Configuración de Ngrok**: Establece tu token de autenticación.
3. **Ejecución**: Inicia el servidor Flask con SocketIO.

In [ ]:
# @title 1. Configuración Inicial y Dependencias
# @markdown Clona el repositorio e instala las librerías necesarias.

import os

# 1.1 Clonar el repositorio (Si no subiste los archivos manualmente)
REPO_URL = "https://github.com/Juanpalojime/FoadsIA.git" # @param {type:"string"}
BRANCH = "main" # @param {type:"string"}

if not os.path.exists("FoadsIA"):
    print(f"[*] Clonando {REPO_URL}...")
    !git clone -b $BRANCH $REPO_URL
else:
    print("[*] Repositorio ya existente. Intentando actualizar...")
    !cd FoadsIA && git pull

# 1.2 Moverse al directorio del backend
if os.path.exists("FoadsIA/backend"):
    os.chdir("FoadsIA/backend")
    print("[*] Directorio de trabajo: backend")
elif os.path.exists("backend"):
    os.chdir("backend")
else:
    print("[!] Advertencia: Asegúrate de estar en la carpeta correcta.")

# 1.3 Instalar dependencias del sistema (FFmpeg)
print("[*] Instalando FFmpeg...")
!apt-get install -y ffmpeg

# 1.4 Instalar dependencias de Python
print("[*] Instalando dependencias de Python... (Esto puede tardar unos minutos)")
!pip install -r requirements.txt
!pip install pyngrok flask-cors flask-socketio eventlet faster-whisper moviepy

In [ ]:
# @title 2. Configurar Ngrok
# @markdown Ingresa tu Authtoken de [Ngrok Dashboard](https://dashboard.ngrok.com/get-started/your-authtoken).
import os
from pyngrok import ngrok, conf

NGROK_TOKEN = "" # @param {type:"string"}

if NGROK_TOKEN:
    print("[*] Configurando Ngrok Token...")
    conf.get_default().auth_token = NGROK_TOKEN
    # Opcional: Configurar región (us, eu, au, ap, sa, jp, in)
    # conf.get_default().region = "us"
else:
    print("[!] Advertencia: No se ha proporcionado NGROK_TOKEN. La conexión podría fallar o caducar rápido.")

In [ ]:
# @title 3. Iniciar Servidor (Auto-Reparable)
# @markdown El servidor se iniciará y mostrará la URL pública de Ngrok.
import os

# --- 1. Lógica de Recuperación ---
if not os.path.exists("FoadsIA") and not os.path.exists("app.py") and not os.path.exists("backend"):
    print("[!] No se detectaron archivos. Clonando repositorio de emergencia...")
    !git clone https://github.com/Juanpalojime/FoadsIA.git

# --- 2. Búsqueda Inteligente de app.py ---
possible_paths = [
    "app.py",
    "FoadsIA/backend/app.py",
    "backend/app.py"
]

target_dir = None
for path in possible_paths:
    if os.path.exists(path):
        target_dir = os.path.dirname(path)
        if target_dir == "": target_dir = "."
        break

# --- 3. Ejecución ---
if target_dir:
    os.chdir(target_dir)
    print(f"[*] Ejecutando desde: {os.getcwd()}")
    
    # Instalación rápida de dependencias si fue un clonado fresco sin paso 1
    try:
        import flask_socketio
    except ImportError:
        print("[*] Instalando dependencias faltantes...")
        !pip install -q flask-socketio pyngrok eventlet faster-whisper moviepy flask-cors

    print("[*] Iniciando servidor...")
    !python app.py
else:
    print("[!] ERROR FATAL: No se encuentra app.py. Revisa el clonado del repositorio.")
    print("Contenido actual:", os.listdir())